# Discover your Reachy robot

This notebook will show you how to start with your Reachy robot and how to start working with it. You will see how you can:

* start your Robot
* read values from the sensor
* send motor commands
* start higher level behaviors

## Start your Reachy robot

**Before starting, you should check in the reachy configuration that the serial port is correctly set and corresponds to your OS!**

You can find where the configuration is located using:

In [7]:
import os
import reachy

print(os.path.join(os.path.dirname(reachy.__file__), 'configuration'))

/Users/pierrerouanet/dev/robot/reachy/software/build/lib/reachy/configuration


There is different configuration here depending on if you are using Leachy, Reachy, with or without hand, etc.

The first step before working with your reachy robot is to setup the connection with the hardware (the motors and sensors). To do that you simply have to run the two following line of codes:

In [ ]:
# Import the reachy library.
from reachy import Reachy

# Setup the communication used to handle the communication with the hardware.
reachy = Reachy()

If everything goes well you should now be connected to your robot. The values are automatically synced between the hardware on your Robot object. **This code will actually check that all motors are connected and correctly configured.** If it's not the case, you will see an error message.

*Beware that you cannot have two instances connected to the same robot at the same time!*

## Use a Leachy instead

You can also do the same for connecting to a Leachy. Simply replace in the following all occurences of *Reachy* by *Leachy* and motor name such as *r_shoulder_roll* by *l_shoulder_roll*.

## Read sensor values

You can simply read values from the motors/sensors. For instance, if you want to display the current position of the shoulder pitch motor you can use the following code:

In [ ]:
print(reachy.shoulder_pitch.present_position)

Note that you can directly access motor by their name:

In [ ]:
print(reachy.arm_yaw)

You can also retrieve all motors using:

In [ ]:
print(reachy.motors)

This can be used to display the current position of all motors:

In [ ]:
for motor in reachy.motors:
    print('The motor "%s" is currently in position: %f' % (motor.name, motor.present_position))

You can similarly access other sensing values like:

* present_position
* present_speed
* present_load
* present_temperature

For instance:

In [ ]:
print(reachy.elbow_pitch.present_speed)

## Send motor commands

Before trying to move Reachy motors you should make sure that they are in stiff mode. Dynamixel motors have two modes:

* compliant: where you can freely move the motors by hand (for instance for demonstration, see below)
* stiff: where the motor is hard and can be controlled

So here we want to change them to stiff mode (actually we will set them to not compliant):

In [ ]:
for motor in reachy.motors:
    motor.compliant = False

You can send motor commands in a similar way. For instance to change the position of the shoulder_roll motor:

In [ ]:
reachy.shoulder_roll.goal_position = 30 # in degrees

*Be careful that the "actual" value is named present_position while the asked one is goal_position*. 

Depending on how you control the motors, the movements can be shaky. A good way to prevent this is to set the maximum speed for a motor:

In [ ]:
reachy.shoulder_roll.moving_speed = 50

Note that in dynamixel motors the maximum speed is named "moving_speed".

In [ ]:
reachy.shoulder_roll.goal_position = 0

The movement should now be smoother.

You can also use the *torque_limit* register (expressed in %) to prevent the motor from forcing too much:

In [ ]:
reachy.shoulder_pitch.torque_limit = 75

## Move from one position to another

Now let's try to move from one position to another one, controlling the speed.

You can write your own way of doing that. You can also check *reachy.goto_position* method.

## Motor loop

You can now try to control one motor with another one...

*Make sure to add some delay in your loop not to overload the background synchronisation loop. Running your loop at 50Hz is always a good value.*

## Higher level behaviors

### Apply sinus and plot the trajectories

The next example shows you how to apply a sinusoid on a motor:

In [ ]:
from time import time, sleep
from numpy import sin, pi

duration = 10 # in sec.
freq = 0.5 # in Hz
amp = 30 # in degrees

update_frequency = 50 # in Hz - meaning how fast we will send motor commands

start = time()
while time() - start < duration:
    target_pos = amp * sin(2 * pi * freq * time())
    reachy.arm_yaw.goal_position = target_pos
    
    sleep(1.0 / update_frequency)

You can modify the example to also record its real and target position during the motion:

In [ ]:
from time import time, sleep
from numpy import sin, pi

duration = 10 # in sec.
freq = 0.5 # in Hz
amp = 30 # in degrees

update_frequency = 50 # in Hz - meaning how fast we will send motor commands

start = time()

real_pos = []
goal_pos = []

while time() - start < duration:
    target_pos = amp * sin(2 * pi * freq * time())
    reachy.arm_yaw.goal_position = target_pos
    
    real_pos.append(reachy.arm_yaw.present_position)
    goal_pos.append(reachy.arm_yaw.goal_position)
    
    sleep(1.0 / update_frequency)

And then plot it using a dedicated library:

In [ ]:
%matplotlib inline

from matplotlib import pylab as plt
from numpy import linspace

t = linspace(0, 10, len(real_pos))

plt.plot(t, real_pos)
plt.plot(t, goal_pos)

plt.legend(('real position', 'goal position'))

You can thus see the lag between the motor command and its reach.

### Using physical demonstrations

As stated above, dynamixel motors have a compliant mode where they can be freely moved. You can change them to compliant mode using:

In [ ]:
for motor in reachy.motors:
    motor.compliant = True

You should now be able to freely move the robot. This can be useful for recording demonstrations. This means you can still record the *present_position* of the motor while they are in compliant mode and then apply these trajectories again when they are in stiff mode to reproduce the demo.

Learning what you have seen before, write a record function that runs for a few seconds and store all motor positions.

Use it to record a simple movement.

You can turn the motor stiff again.

You can now write a simple function to replay your recorded movement.

You have to be especially careful here:
- not to enter dead zone when recording
- maybe set a maximum speed to avoid high acceleration
- also make sure to first reach the starting point of a recorded movement before re-playing. Otherwise, you will observe a very fast motion toward the beginning of your trajectory.

Reachy also provides a more convenient way to record and replay movement. Try looking in the source code if you find anything [hint](https://github.com/pollen-robotics/reachy/tree/6e5439e6c7c3315df6eb9f28e55d227194fd5f9f/software/reachy/primitives). Primitives area actually a good way to implement this kind og behavior. More info can be found in pypot's documentation: http://poppy-project.github.io/pypot/primitive.html

## Going further

### Simulation

Try to control a Reachy or a Leachy in simulation.

## Tele-operation

Let's try controling a Reachy with a Leachy... 

You should be really careful:

- to motor orientation
- not to use too fast motion
- remember compliant vs stiff mode